In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import sys
import os

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
other_folder = os.path.join(parent_dir, 'app')
sys.path.append(other_folder)

In [4]:
from lib import *

Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.

Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.



In [5]:
shop_sales = pd.read_csv('../app/test_data/shop_sales.csv')
shop_sales_dates = pd.read_csv('../app/test_data/shop_sales_dates.csv')
shop_sales_prices = pd.read_csv('../app/test_data/shop_sales_prices.csv')

In [6]:
df = merge_files_to_dataset(shop_sales, shop_sales_prices, shop_sales_dates)

In [7]:
df = summ_sales_data(df, 'date', granularity = 'Day')

In [8]:
df = df_encoding(df)

In [9]:
df['date'].max()

Timestamp('2016-01-21 00:00:00')

In [10]:
df['item_id'].unique()

array(['STORE_2_085', 'STORE_2_043', 'STORE_2_054', 'STORE_2_325',
       'STORE_2_090', 'STORE_2_252', 'STORE_2_376', 'STORE_2_391',
       'STORE_2_524', 'STORE_2_555', 'STORE_2_586', 'STORE_2_587',
       'STORE_2_681', 'STORE_2_714', 'STORE_2_716', 'STORE_3_012',
       'STORE_3_114', 'STORE_3_357', 'STORE_3_360', 'STORE_3_090',
       'STORE_3_169', 'STORE_3_222', 'STORE_3_252', 'STORE_3_358',
       'STORE_3_389', 'STORE_3_555', 'STORE_3_586', 'STORE_3_587',
       'STORE_3_714', 'STORE_3_804', 'STORE_1_064', 'STORE_1_065',
       'STORE_1_325', 'STORE_1_376', 'STORE_1_090', 'STORE_1_252',
       'STORE_1_339', 'STORE_1_546', 'STORE_1_547', 'STORE_1_555',
       'STORE_1_584', 'STORE_1_586', 'STORE_1_587', 'STORE_1_714',
       'STORE_1_727'], dtype=object)

In [11]:
data_prediction, model, horizont = get_preds(df, ['STORE_1_584'], 2)

запуск первого шага
мы сошлись
[]
10000000000



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
data_prediction

,cnt_SMA_3_lag_2,cnt_WMA_3_lag_2,cnt_EMA_3_lag_2,mean,item_id,date,cnt,model_prediction,arima_prediction
0,0.333333,0.333333,0.607637,1.25,STORE_1_584,2015-12-17,0.0,2.962588,1.506920
1,0.666667,0.666667,0.803819,1.25,STORE_1_584,2015-12-18,1.0,3.115803,1.403077
2,0.333333,0.333333,0.401909,1.25,STORE_1_584,2015-12-19,1.0,2.962588,1.399992
3,0.666667,0.666667,0.700955,1.25,STORE_1_584,2015-12-20,1.0,2.962588,1.377599
4,0.666667,0.833333,0.850477,1.25,STORE_1_584,2015-12-21,2.0,2.968484,1.356460
5,1.000000,1.000000,0.925239,1.25,STORE_1_584,2015-12-22,0.0,2.876810,1.411968
6,1.333333,1.500000,1.462619,1.25,STORE_1_584,2015-12-23,0.0,3.030980,1.293960
7,1.000000,0.833333,0.731310,1.25,STORE_1_584,2015-12-24,0.0,2.962588,1.221520
8,0.666667,0.333333,0.365655,1.25,STORE_1_584,2015-12-25,0.0,2.962588,1.153136
9,0.000000,0.000000,0.182827,1.25,STORE_1_584,2015-12-26,2.0,3.016293,1.088579


In [13]:
result = evaluate_predictions(data_prediction, horizont)

In [14]:
result

,Prediction,MAPE,MAE,RMSE
0,model_prediction,131.563939,1.587945,1.819629
1,cnt_SMA_3_lag_2,54.761905,1.125000,1.662495
2,mean,36.842758,0.916667,1.527525
3,arima_prediction,38.300528,0.948841,1.569897
